Loading and creating the mme_dataset

In [2]:
import pandas as pd

# create a dataframe with only the ExtendedSessionID column from the 71M SharedResponses.csv file

reader2 = pd.read_csv('SharedResponses.csv', usecols=['ResponseID'])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [2]:
# code that will check if there are two of them in the dataset

# Count how many times each ResponseID appears
response_counts = reader2['ResponseID'].value_counts()

# Get the ResponseIDs that appear exactly twice
ids_to_keep = response_counts[response_counts == 2].index

# Filter the rows where ResponseID is in the list of ids_to_keep
complete_responseid = reader2[reader2['ResponseID'].isin(ids_to_keep)]


In [3]:
complete_responseid.value_counts().min()

2

In [4]:
complete_responseid.shape

(67907006, 1)

In [5]:
# Drop duplicates based on 'ResponseID' and keep the first occurrence
complete_responseid = complete_responseid.drop_duplicates(subset=['ResponseID'], keep='first')

In [6]:
reader_subset = complete_responseid.head(8500000)  # want 17M rows (with accounting for NAn's and deleting rows with 'random' (around 10%)), so 17M / 2 = 8.5M ResponseID's necessary

# taking the top 8,5 Million, as I assume the data in the set is random enough to not have a bias (like country) in the first 8,5M rows

In [7]:
# checking if they are all unique

reader_subset.nunique()

ResponseID    8500000
dtype: int64

In [8]:
# transform to a list to feed to the for loop that will extract all the corresponding columns from the SharedResponses.csv file

reader_sub_list = reader_subset['ResponseID'].tolist()

In [9]:
# check if it went okay
print(len(reader_sub_list))     # should be 8500000

8500000


In [10]:
# empty dataframe to append the rows of the sessions with 24 rows to

subset = pd.DataFrame()

In [11]:
# reading SharedResponses to extract the rows with ResponseID's in reader_sub_list
# this will result in a dataframe 'subset' that contains around 17 million rows (8,5 * 2)

chunk_size = 500_000
reader = pd.read_csv('SharedResponses.csv', chunksize=chunk_size, dtype=str, low_memory=False)

for i, chunk in enumerate(reader):
    
    print(f"Processing chunk {i+1}")

    # Filter rows where ResponseID is in reader_subset
    subset_chunk = chunk[chunk['ResponseID'].isin(reader_sub_list)]

    # Append filtered chunk to empty df
    subset = pd.concat([subset, subset_chunk], ignore_index=True)

    print(f"Finished processing chunk {i+1}")

print("All chunks have been processed and combined.")

Processing chunk 1
Finished processing chunk 1
Processing chunk 2
Finished processing chunk 2
Processing chunk 3
Finished processing chunk 3
Processing chunk 4
Finished processing chunk 4
Processing chunk 5
Finished processing chunk 5
Processing chunk 6
Finished processing chunk 6
Processing chunk 7
Finished processing chunk 7
Processing chunk 8
Finished processing chunk 8
Processing chunk 9
Finished processing chunk 9
Processing chunk 10
Finished processing chunk 10
Processing chunk 11
Finished processing chunk 11
Processing chunk 12
Finished processing chunk 12
Processing chunk 13
Finished processing chunk 13
Processing chunk 14
Finished processing chunk 14
Processing chunk 15
Finished processing chunk 15
Processing chunk 16
Finished processing chunk 16
Processing chunk 17
Finished processing chunk 17
Processing chunk 18
Finished processing chunk 18
Processing chunk 19
Finished processing chunk 19
Processing chunk 20
Finished processing chunk 20
Processing chunk 21
Finished processin

In [12]:
subset.shape

(17000000, 41)

In [13]:
subset_csv = subset.to_csv('subset.csv', index=False)

In [1]:
import pandas as pd
df_mme = pd.read_csv('subset.csv')

C:\Users\esmku\AppData\Local\Temp\ipykernel_27732\2060601064.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mme = pd.read_csv('subset.csv')


In [2]:
df_mme.shape

(17000000, 41)

In [3]:
df_mme.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
0,2222bRQqBTZ6dLnPH,32757157_6999801415950060.0,6.999801e+15,7,0,0,0,1,Fit,Fitness,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
1,2222sJk4DcoqXXi98,1043988516_3525281295.0,3.525281e+09,2,0,0,0,0,Rand,Random,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2223CNmvTr2Coj4wp,-1613944085_422160228641876.0,4.221602e+14,10,0,1,0,1,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2223Xu54ufgjcyMR3,1425316635_327833569077076.0,3.278336e+14,11,0,0,1,0,Old,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2223jMWDEGNeszivb,-1683127088_785070916172117.0,7.850709e+14,8,0,1,0,2,More,Utilitarian,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [4]:
df_mme['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      3050188
Species          3002434
Age              2998622
Fitness          2995878
Gender           2992726
Random           1494764
Social Status     465388
Name: count, dtype: int64

In [5]:
# removing all rows with 'random' in the ScenarioTypeStrict column

df_mme = df_mme[df_mme['ScenarioTypeStrict'] != 'Random']

In [6]:
df_mme.shape # should be 17000000 - 1494764 = 15505236

(15505236, 41)

In [7]:
df_mme.isna().sum()

ResponseID                       0
ExtendedSessionID                0
UserID                        1926
ScenarioOrder                    0
Intervention                     0
PedPed                           0
Barrier                          0
CrossingSignal                   0
AttributeLevel                   0
ScenarioTypeStrict               0
ScenarioType                     0
DefaultChoice               492034
NonDefaultChoice            492034
DefaultChoiceIsOmission     492034
NumberOfCharacters               0
DiffNumberOFCharacters           0
Saved                            0
Template                   2026082
DescriptionShown           2026082
LeftHand                   2026082
UserCountry3                145768
Man                              0
Woman                            0
Pregnant                         0
Stroller                         0
OldMan                           0
OldWoman                         0
Boy                              0
Girl                

In [8]:
# deleting rows with NaN's in the UserID column

df_mme = df_mme.dropna(subset=['UserID'])

In [9]:
df_mme.shape # should be 15505236 - 1926 = 15503310

(15503310, 41)

In [10]:
# rows that are needed to make dataset 2% llm and 98% human
total_rows_dataset = 293568 / 2 * 98
print(total_rows_dataset)    
rows_to_delete = df_mme.shape[0] - total_rows_dataset
print(rows_to_delete) 

# need to delete 1118478 rows 

14384832.0
1118478.0


In [19]:
df_mme.shape

(15503310, 41)

In [12]:
# randomly delete 1118478 / 2 = 559239 unique ResponseID's (is 1118478 rows), to ensure 2% of the dataset is LLMs and 98% humans

# Getting unique UserIDs
Response_unique = df_mme['ResponseID'].unique()
print(len(Response_unique))  # should be 15503310/2 = 7751655

# Selecting 567,723 UserIDs from the unique set
Response_delete = pd.Series(Response_unique).sample(n=559239, random_state=42)


7751655


In [29]:
df_mme.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
0,2222bRQqBTZ6dLnPH,32757157_6999801415950060.0,6.999801e+15,7,0,0,0,1,Fit,Fitness,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
2,2223CNmvTr2Coj4wp,-1613944085_422160228641876.0,4.221602e+14,10,0,1,0,1,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2223Xu54ufgjcyMR3,1425316635_327833569077076.0,3.278336e+14,11,0,0,1,0,Old,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2223jMWDEGNeszivb,-1683127088_785070916172117.0,7.850709e+14,8,0,1,0,2,More,Utilitarian,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,2224H2QBFKNsMmRQc,1661661891_4304873273230329.0,4.304873e+15,11,0,1,0,0,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [33]:
df_mme_98 = df_mme[~df_mme['ResponseID'].isin(Response_delete)]

In [34]:
df_mme_98.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
2,2223CNmvTr2Coj4wp,-1613944085_422160228641876.0,4.221602e+14,10,0,1,0,1,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2224H2QBFKNsMmRQc,1661661891_4304873273230329.0,4.304873e+15,11,0,1,0,0,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6,2224YxTZcu4sJqTSD,-887960483_174929057557052.0,1.749291e+14,6,0,0,0,2,Hoomans,Species,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
7,2224g4ytARX4QT5rB,213978760_9992828917431898.0,9.992829e+15,7,0,0,1,0,Less,Utilitarian,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,2224kBG72574tbZD3,737909459_839962439872333.0,8.399624e+14,12,0,0,1,0,Old,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# check what the distribution of RepsonseID's is (kinda)
# check if the indices of a responseID here are the same after transforming the ResponseID column
# if the indices are the same means ResponseID column is succesfully transformed

indices = df_mme_98[df_mme_98["ResponseID"] == '2223CNmvTr2Coj4wp'].index
print(indices)

Index([2, 8500002], dtype='int64')


In [36]:
# checking shape of df_filtered

df_mme_98.shape   # should be 15503310 - 1118478 = 14384832

(14384832, 41)

In [37]:
# Changing the responseID

# Define the starting point for the new ResponseID
starting_id = 146784

# Step 1: Get the unique ResponseIDs
unique_response_ids = df_mme_98['ResponseID'].unique()

# Step 2: Create a mapping from old ResponseID to new 'res_' formatted ID
response_id_mapping = {old_id: f'res_{i:08d}' for i, old_id in enumerate(unique_response_ids, starting_id)}

# Step 3: Replace the original ResponseID with the new mapped IDs
df_mme_98['ResponseID'] = df_mme_98['ResponseID'].map(response_id_mapping)

C:\Users\esmku\AppData\Local\Temp\ipykernel_27732\3824585436.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mme_98['ResponseID'] = df_mme_98['ResponseID'].map(response_id_mapping)


In [38]:
# ResponseID starts with res_00146784
df_mme_98.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
2,res_00146784,-1613944085_422160228641876.0,4.221602e+14,10,0,1,0,1,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,res_00146785,1661661891_4304873273230329.0,4.304873e+15,11,0,1,0,0,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6,res_00146786,-887960483_174929057557052.0,1.749291e+14,6,0,0,0,2,Hoomans,Species,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
7,res_00146787,213978760_9992828917431898.0,9.992829e+15,7,0,0,1,0,Less,Utilitarian,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,res_00146788,737909459_839962439872333.0,8.399624e+14,12,0,0,1,0,Old,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
indices = df_mme_98[df_mme_98["ResponseID"] == 'res_00146784'].index
print(indices)

# checking if the indices are the same after transforming the ResponseID column
# compared to the indices of the ResponseID before transforming the column - they are!
# both are [2, 8500002]

Index([2, 8500002], dtype='int64')


In [40]:
df_mme_98.tail()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
16999993,res_07339195,-388280772_2722777491.0,2.722777e+09,12,1,1,0,0,Less,Utilitarian,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16999995,res_07339196,1658771898_8006215139757842.0,8.006215e+15,8,1,0,0,2,Fat,Fitness,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16999997,res_07339197,1269164515_3453341116.0,3.453341e+09,9,1,0,0,0,Male,Gender,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16999998,res_07339198,-1589392172_4126242148.0,4.126242e+09,3,1,1,0,1,Fat,Fitness,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16999999,res_07339199,-1014371005_411438207332712.0,4.114382e+14,10,1,0,0,0,More,Utilitarian,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [41]:
df_mme_98.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder',
       'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel',
       'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
       'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
       'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
       'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
       'Cat'],
      dtype='object')

In [42]:
# already deleting some columns to ensure it fits in memory
# deleting the columns that are not necessary for the modelling: ExtendedSessionID, DefaultChoice, NonDefaultChoice, DefaultChoiceIsOmission, Template

df_mme_clean = df_mme_98.drop(columns=['ExtendedSessionID', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'Template'])

In [43]:
df_mme_clean.shape # should be 14367864 rows and 41-5 = 36 columns

(14384832, 36)

In [1]:
# resetting the index
df_mme_clean = df_mme_clean.reset_index(drop=True)

NameError: name 'df_mme_clean' is not defined

In [45]:
# saving this pre-preprocessed mme dataset to a csv file

df_mme_clean.to_csv('mme_dataset.csv', index=False)